In [1]:
import pandas as pd
import os
from pathlib import Path
from sklearn.model_selection import train_test_split

In [2]:
filted_fitzpatrick_df = pd.read_csv("../Fitzpatrick17k_current_files/fitzpatrick17k_downloaded.csv")
ddi_df = pd.read_csv("../ddidiversedermatologyimages/ddi_metadata.csv")

In [3]:
filted_fitzpatrick_df.head()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,url,url_alphanum
0,5e82a45bc5d78bd24ae9202d194423f8,3,3,drug induced pigmentary changes,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmminoc...
1,fa2911a9b13b6f8af79cb700937cc14f,1,1,photodermatoses,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicpphoto...
2,d2bac3c9e4499032ca8e9b07c7d3bc40,2,3,dermatofibroma,benign dermal,benign,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicdderma...
3,0a94359e7eaacd7178e06b2823777789,1,1,psoriasis,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppsori...
4,a39ec3b1f22c08a421fa20535e037bba,1,1,psoriasis,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppsori...


In [4]:
# use fitzpatrick_scale as skin tone (current plan)
filted_fitzpatrick_df["fitzpatrick_scale"].unique()

array([ 3,  1,  2,  6,  4, -1,  5])

In [5]:
# drop -1 skin tone
filted_fitzpatrick_df = filted_fitzpatrick_df[(filted_fitzpatrick_df["fitzpatrick_scale"] >= 0)].copy()

In [6]:
filted_fitzpatrick_df = filted_fitzpatrick_df[filted_fitzpatrick_df["three_partition_label"] != "non-neoplastic"]

In [7]:
len(filted_fitzpatrick_df[filted_fitzpatrick_df["three_partition_label"] == "non-neoplastic"])

0

In [8]:
len(filted_fitzpatrick_df)

4316

In [9]:
filted_fitzpatrick_df["three_partition_label"].unique()

array(['benign', 'malignant'], dtype=object)

In [10]:
# filted_fitzpatrick_df = filted_fitzpatrick_df.dropna(subset=["fitzpatrick_scale"]).copy()
# filted_fitzpatrick_df = filted_fitzpatrick_df.dropna(subset=["three_partition_label"]).copy()

In [11]:
filted_fitzpatrick_df.tail()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,url,url_alphanum
16495,c558af11b54f8c7789a2a0b0bff1826f,3,3,basal cell carcinoma,malignant epidermal,malignant,NaN,http://atlasdermatologico.com.br/img?imageId=532,httpwwwatlasdermatologicocombrimgimageId532.jpg
16497,67779d244d362f8c4f089a5b4d4fc989,4,-1,squamous cell carcinoma,malignant epidermal,malignant,NaN,http://atlasdermatologico.com.br/img?imageId=8691,httpwwwatlasdermatologicocombrimgimageId8691.jpg
16504,8eb074ec464733f3a583af942fee9a14,4,3,prurigo nodularis,benign epidermal,benign,1 Diagnostic,http://atlasdermatologico.com.br/img?imageId=6135,httpwwwatlasdermatologicocombrimgimageId6135.jpg
16506,218ea232ca4b0255a395886ee86cbada,3,2,basal cell carcinoma,malignant epidermal,malignant,NaN,http://atlasdermatologico.com.br/img?imageId=586,httpwwwatlasdermatologicocombrimgimageId586.jpg
16510,5807f27e6324eb4884a8ede5aa1cc9ba,3,2,porokeratosis actinic,benign epidermal,benign,NaN,http://atlasdermatologico.com.br/img?imageId=5914,httpwwwatlasdermatologicocombrimgimageId5914.jpg


In [12]:
ddi_df.head()

,Unnamed: 0,DDI_ID,DDI_file,skin_tone,malignant,disease
0,0,1,000001.png,56,True,melanoma-in-situ
1,1,2,000002.png,56,True,melanoma-in-situ
2,2,3,000003.png,56,True,mycosis-fungoides
3,3,4,000004.png,56,True,squamous-cell-carcinoma-in-situ
4,4,5,000005.png,12,True,basal-cell-carcinoma


In [13]:
ddi_df["malignant"] = (ddi_df["malignant"]).astype(int)

In [14]:
ddi_df.head()

,Unnamed: 0,DDI_ID,DDI_file,skin_tone,malignant,disease
0,0,1,000001.png,56,1,melanoma-in-situ
1,1,2,000002.png,56,1,melanoma-in-situ
2,2,3,000003.png,56,1,mycosis-fungoides
3,3,4,000004.png,56,1,squamous-cell-carcinoma-in-situ
4,4,5,000005.png,12,1,basal-cell-carcinoma


In [15]:
filted_fitzpatrick_df["malignant"] = (filted_fitzpatrick_df["three_partition_label"] == "malignant").astype(int)

In [16]:
# add new column to fitzpatrick, align with ddi
skin_tone_map = {
    1: 12, 2: 12,
    3: 34, 4: 34,
    5: 56, 6: 56,
}

filted_fitzpatrick_df["skin_tone"] = filted_fitzpatrick_df["fitzpatrick_scale"].map(skin_tone_map).astype(int)
# filted_df["skin_tone"] = (filted_df["skin_tone"]).astype(int)

In [17]:
filted_fitzpatrick_df.head()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,url,url_alphanum,malignant,skin_tone
2,d2bac3c9e4499032ca8e9b07c7d3bc40,2,3,dermatofibroma,benign dermal,benign,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicdderma...,0,12
5,45f7fe0e10214e32e890cad9d29d4811,6,5,kaposi sarcoma,malignant dermal,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpickkapos...,1,56
13,b87804452f60aa162a6d29c0f66a2466,2,1,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicLlmmlm...,1,12
18,4c3f795cf8eb72b946f9bd2642cf23c1,6,5,melanoma,malignant melanoma,malignant,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmmelan...,1,56
19,99247c9fe486aa9ab71686c8e676c135,2,1,dermatofibroma,benign dermal,benign,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicdderma...,0,12


In [18]:
ddi_df.head()

,Unnamed: 0,DDI_ID,DDI_file,skin_tone,malignant,disease
0,0,1,000001.png,56,1,melanoma-in-situ
1,1,2,000002.png,56,1,melanoma-in-situ
2,2,3,000003.png,56,1,mycosis-fungoides
3,3,4,000004.png,56,1,squamous-cell-carcinoma-in-situ
4,4,5,000005.png,12,1,basal-cell-carcinoma


## Train

In [19]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

In [20]:
# train validation split

df = filted_fitzpatrick_df.copy()
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["malignant"],
    random_state=543
)
print("train:", len(train_df), "val:", len(val_df))

train: 3452 val: 864


In [21]:
class FitzDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = "../Fitzpatrick17k_current_files/Fitzpatrick17k_images/" + row["md5hash"] + ".jpg"
        img = Image.open(img_path).convert("RGB")
        
        if self.transform is not None:
            img = self.transform(img)
        label = int(row["malignant"])
        return img, label


image_size = 224
basic_tf = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
])

batch_size = 32

train_loader = DataLoader(
    FitzDataset(train_df, transform=basic_tf),
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
)

val_loader = DataLoader(
    FitzDataset(val_df, transform=basic_tf),
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
)

In [22]:
import torch
import torch.nn as nn
from torchvision import models
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

Using device: cuda


In [24]:
def evaluate(model, loader, device):
    model.eval()
    all_probs = []
    all_labels = []
    for imgs, labels in loader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        logits = model(imgs)
        probs = torch.softmax(logits, dim=1)[:, 1]
        # all_probs.append(probs.cpu().numpy())
        # all_labels.append(labels.cpu().numpy())

        all_probs.append(probs.detach().cpu().numpy())
        all_labels.append(labels.detach().cpu().numpy())
        
    all_probs = np.concatenate(all_probs)
    all_labels = np.concatenate(all_labels)

    y_pred = (all_probs >= 0.5).astype(int)
    acc = accuracy_score(all_labels, y_pred)
    try:
        auc = roc_auc_score(all_labels, all_probs)
    except ValueError:
        auc = float("nan")
    return {"acc": acc, "auc": auc}

In [25]:
import time

num_epochs = 10

best_auc = -1.0

for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0
    n_batches = 0

    start_time = time.time()
    for batch_idx, (imgs, labels) in enumerate(train_loader):
    # for imgs, labels in train_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        logits = model(imgs)
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        n_batches += 1

        if (batch_idx + 1) % 20 == 0:
            elapsed = time.time() - start_time
            print(f"  batch {batch_idx+1}, elapsed {elapsed:.1f}s")
            start_time = time.time()

    train_loss = running_loss / max(n_batches, 1)
    val_metrics = evaluate(model, val_loader, device)

    print(f"[Epoch {epoch}] "
          f"train_loss={train_loss:.4f} "
          f"val_acc={val_metrics['acc']:.4f} "
          f"val_auc={val_metrics['auc']:.4f}")

    if val_metrics["auc"] > best_auc:
        best_acc = val_metrics["acc"]
        best_auc = val_metrics["auc"]
        torch.save(model.state_dict(), "resnet18_fitz_basic.pt")
        print(f"  -> Save new best model，AUC={best_auc:.4f}, ACC={best_acc:.4f}")

  batch 20, elapsed 10.2s
  batch 40, elapsed 8.0s
  batch 60, elapsed 6.8s
  batch 80, elapsed 6.7s
  batch 100, elapsed 7.0s
[Epoch 1] train_loss=0.6929 val_acc=0.7141 val_auc=0.7844
  -> Save new best model，AUC=0.7844, ACC=0.7141
  batch 20, elapsed 9.9s
  batch 40, elapsed 9.1s
  batch 60, elapsed 12.5s
  batch 80, elapsed 8.5s
  batch 100, elapsed 9.0s
[Epoch 2] train_loss=0.5805 val_acc=0.6759 val_auc=0.7633
  batch 20, elapsed 8.8s
  batch 40, elapsed 9.2s
  batch 60, elapsed 10.5s
  batch 80, elapsed 11.2s
  batch 100, elapsed 11.3s
[Epoch 3] train_loss=0.5531 val_acc=0.7234 val_auc=0.7830
  batch 20, elapsed 13.5s
  batch 40, elapsed 14.3s
  batch 60, elapsed 11.3s
  batch 80, elapsed 9.4s
  batch 100, elapsed 8.6s
[Epoch 4] train_loss=0.4928 val_acc=0.7049 val_auc=0.7653
  batch 20, elapsed 8.8s
  batch 40, elapsed 9.4s
  batch 60, elapsed 9.0s
  batch 80, elapsed 8.8s
  batch 100, elapsed 9.3s
[Epoch 5] train_loss=0.4730 val_acc=0.7257 val_auc=0.7956
  -> Save new best model

## Test on DDI

In [26]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms


DDI_IMG_DIR = Path("../ddidiversedermatologyimages")


ddi_df["img_path"] = ddi_df["DDI_file"].apply(lambda fn: str(DDI_IMG_DIR / fn))


from pathlib import Path
ddi_df = ddi_df[ddi_df["img_path"].apply(lambda p: Path(p).is_file())].copy()
print("DDI counts：", len(ddi_df))


image_size = 224
basic_tf = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
])

class DdiDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row["img_path"]).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        label = int(row["malignant"])
        tone = row["skin_tone"]
        return img, label, tone

ddi_loader = DataLoader(
    DdiDataset(ddi_df, transform=basic_tf),
    batch_size=64,
    shuffle=False,
    num_workers=0,
)

DDI counts： 656


In [27]:
import torch
import torch.nn as nn
from torchvision import models
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)
model.to(device)

ckpt_path = "resnet18_fitz_basic.pt"
state_dict = torch.load(ckpt_path, map_location=device)
model.load_state_dict(state_dict)
model.eval()
print("Loaded checkpoint from:", ckpt_path)

Using device: cuda
Loaded checkpoint from: resnet18_fitz_basic.pt


In [28]:
def eval_on_ddi(model, loader, device):
    model.eval()
    all_probs = []
    all_labels = []
    all_tones = []

    for imgs, labels, tones in loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        logits = model(imgs)
        probs = torch.softmax(logits, dim=1)[:, 1]

        all_probs.append(probs.detach().cpu().numpy())
        all_labels.append(labels.detach().cpu().numpy())
        all_tones.append(tones.numpy())

    all_probs = np.concatenate(all_probs)
    all_labels = np.concatenate(all_labels)
    all_tones = np.concatenate(all_tones)

    # overall
    y_pred = (all_probs >= 0.5).astype(int)
    acc = accuracy_score(all_labels, y_pred)
    f1 = f1_score(all_labels, y_pred)
    try:
        auc = roc_auc_score(all_labels, all_probs)
    except ValueError:
        auc = float("nan")

    print("=== Overall DDI ===")
    print(f"ACC = {acc:.4f}, F1 = {f1:.4f}, AUC = {auc:.4f}")

    for tone_value in [12, 34, 56]:
        mask = (all_tones == tone_value)
        if mask.sum() == 0:
            continue
        probs_t = all_probs[mask]
        labels_t = all_labels[mask]
        y_pred_t = (probs_t >= 0.5).astype(int)
        acc_t = accuracy_score(labels_t, y_pred_t)
        f1_t = f1_score(labels_t, y_pred_t)
        try:
            auc_t = roc_auc_score(labels_t, probs_t)
        except ValueError:
            auc_t = float("nan")
        print(f"--- skin_tone:{tone_value} ---")
        print(f"  n = {mask.sum()}, ACC = {acc_t:.4f}, F1 = {f1_t:.4f}, AUC = {auc_t:.4f}")

eval_on_ddi(model, ddi_loader, device)

=== Overall DDI ===
ACC = 0.6753, F1 = 0.3364, AUC = 0.6121
--- skin_tone:12 ---
  n = 208, ACC = 0.6250, F1 = 0.3500, AUC = 0.5984
--- skin_tone:34 ---
  n = 241, ACC = 0.6846, F1 = 0.3871, AUC = 0.7003
--- skin_tone:56 ---
  n = 207, ACC = 0.7150, F1 = 0.2338, AUC = 0.5193


In [29]:
ddi_df["malignant"].value_counts()
# ddi_df["malignant"].mean()

malignant
0    485
1    171
Name: count, dtype: int64

In [30]:
ddi_df["malignant"].mean()

np.float64(0.2606707317073171)